# Black Scholes Functions

In [1]:
import numpy as np
from scipy.stats import norm
N = norm.cdf

def bs_call(S, K, T, r, vol):
    d1 = (np.log(S/K) + (r + 0.5*vol**2)*T) / (vol*np.sqrt(T))
    d2 = d1 - vol * np.sqrt(T)
    return S * norm.cdf(d1) - np.exp(-r * T) * K * norm.cdf(d2)

def bs_vega(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    return S * norm.pdf(d1) * np.sqrt(T)

def find_vol(target_value, S, K, T, r, *args):
    MAX_ITERATIONS = 200
    PRECISION = 1.0e-5
    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = bs_call(S, K, T, r, sigma)
        vega = bs_vega(S, K, T, r, sigma)
        diff = target_value - price  # our root
        if (abs(diff) < PRECISION):
            return sigma
        sigma = sigma + diff/vega # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

In [2]:
S = 100
K = 100
T = 11
r = 0.01
vol = 0.25

V_market = bs_call(S, K, T, r, vol)
implied_vol = find_vol(V_market, S, K, T, r)

print ('Implied vol: %.2f%%' % (implied_vol * 100))
print ('Market price = %.2f' % V_market)
print ('Model price = %.2f' % bs_call(S, K, T, r, implied_vol))

Implied vol: 25.00%
Market price = 35.94
Model price = 35.94


# Import Data From RobinHood

In [10]:
import robin_stocks as r
import pandas as pd
from datetime import datetime
# from config import *
import numpy as np

In [19]:
data = r.options.find_tradable_options('AMD', optionType='call')
df = pd.DataFrame(data)
df.head()

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...


,chain_id,chain_symbol,created_at,expiration_date,id,issue_date,min_ticks,rhs_tradability,state,strike_price,tradability,type,updated_at,url,sellout_datetime
0,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481180Z,2021-02-19,95e5c49d-07f7-4117-9886-4d11f71ea73b,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,95.0000,tradable,call,2020-11-06T01:24:57.481186Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
1,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481141Z,2021-02-19,5d288654-8782-4f76-bfb4-4944d1eb186a,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,90.0000,tradable,call,2020-11-06T01:24:57.481146Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
2,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481102Z,2021-02-19,8ede0366-ecb0-4f9d-9529-c3d30e4b2f81,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,85.0000,tradable,call,2020-11-06T01:24:57.481107Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
3,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480944Z,2021-02-19,674d82ab-6756-443e-b1e2-64b02e1764f0,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,75.0000,tradable,call,2020-11-06T01:24:57.480950Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
4,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480866Z,2021-02-19,a28cfa69-c4bb-4763-9f08-35a39eb5aec0,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,80.0000,tradable,call,2020-11-06T01:24:57.480871Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00


In [20]:
df.columns

Index(['chain_id', 'chain_symbol', 'created_at', 'expiration_date', 'id',
       'issue_date', 'min_ticks', 'rhs_tradability', 'state', 'strike_price',
       'tradability', 'type', 'updated_at', 'url', 'sellout_datetime'],
      dtype='object')

In [23]:
dfShort = df.head(10)


,chain_id,chain_symbol,created_at,expiration_date,id,issue_date,min_ticks,rhs_tradability,state,strike_price,tradability,type,updated_at,url,sellout_datetime
0,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481180Z,2021-02-19,95e5c49d-07f7-4117-9886-4d11f71ea73b,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,95.0000,tradable,call,2020-11-06T01:24:57.481186Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
1,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481141Z,2021-02-19,5d288654-8782-4f76-bfb4-4944d1eb186a,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,90.0000,tradable,call,2020-11-06T01:24:57.481146Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
2,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481102Z,2021-02-19,8ede0366-ecb0-4f9d-9529-c3d30e4b2f81,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,85.0000,tradable,call,2020-11-06T01:24:57.481107Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
3,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480944Z,2021-02-19,674d82ab-6756-443e-b1e2-64b02e1764f0,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,75.0000,tradable,call,2020-11-06T01:24:57.480950Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
4,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480866Z,2021-02-19,a28cfa69-c4bb-4763-9f08-35a39eb5aec0,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,80.0000,tradable,call,2020-11-06T01:24:57.480871Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
5,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480748Z,2021-02-19,4e66bec9-35cd-40d0-994c-705b02c399b5,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,105.0000,tradable,call,2020-11-06T01:24:57.480754Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
6,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480709Z,2021-02-19,b0dde31e-529d-4298-b95e-22b98c189842,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,70.0000,tradable,call,2020-11-06T01:24:57.480715Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
7,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480670Z,2021-02-19,4c14c134-86bd-47fe-9eda-56f03edcd1ff,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,60.0000,tradable,call,2020-11-06T01:24:57.480675Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
8,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480630Z,2021-02-19,dc19d215-1a5d-4ad3-acd1-be58842f605f,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,77.5000,tradable,call,2020-11-06T01:24:57.480636Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00
9,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480591Z,2021-02-19,f5c0f0af-5806-4988-b84e-a88d1c944bce,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,87.5000,tradable,call,2020-11-06T01:24:57.480597Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00


In [31]:
strikeList = list(dfShort['strike_price'])
strikeList = [float(i) for i in strikeList]


[95.0, 90.0, 85.0, 75.0, 80.0, 105.0, 70.0, 60.0, 77.5, 87.5]

In [34]:
# test it out 

S = 100
K = 100
T = 11
r = 0.01
vol = 0.25

bs_call(95, K, T, r, vol)

32.45478451161509

# Calculate Implied Volatility for Every Stike Price 

In [48]:
vMarketList = [ bs_call(i, K, T, r, vol) for i in strikeList]
ivList = []
for i in range(len(vMarketList)):
    ivList.append(find_vol(vMarketList[i], strikeList[i], K, T, r))
ivList

[0.24999993374685453,
 0.24999992378186947,
 0.24999993182822486,
 0.24999998356759362,
 0.24999995602170305,
 0.24999997636507984,
 0.24999999866868236,
 0.25000000916483783,
 0.24999997046617048,
 0.24999992539995045]

In [49]:
dfShort['IV'] = ivList
dfShort

C:\Users\ashformr\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,chain_id,chain_symbol,created_at,expiration_date,id,issue_date,min_ticks,rhs_tradability,state,strike_price,tradability,type,updated_at,url,sellout_datetime,IV
0,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481180Z,2021-02-19,95e5c49d-07f7-4117-9886-4d11f71ea73b,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,95.0000,tradable,call,2020-11-06T01:24:57.481186Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
1,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481141Z,2021-02-19,5d288654-8782-4f76-bfb4-4944d1eb186a,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,90.0000,tradable,call,2020-11-06T01:24:57.481146Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
2,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.481102Z,2021-02-19,8ede0366-ecb0-4f9d-9529-c3d30e4b2f81,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,85.0000,tradable,call,2020-11-06T01:24:57.481107Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
3,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480944Z,2021-02-19,674d82ab-6756-443e-b1e2-64b02e1764f0,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,75.0000,tradable,call,2020-11-06T01:24:57.480950Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
4,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480866Z,2021-02-19,a28cfa69-c4bb-4763-9f08-35a39eb5aec0,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,80.0000,tradable,call,2020-11-06T01:24:57.480871Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
5,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480748Z,2021-02-19,4e66bec9-35cd-40d0-994c-705b02c399b5,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,105.0000,tradable,call,2020-11-06T01:24:57.480754Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
6,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480709Z,2021-02-19,b0dde31e-529d-4298-b95e-22b98c189842,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,70.0000,tradable,call,2020-11-06T01:24:57.480715Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
7,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480670Z,2021-02-19,4c14c134-86bd-47fe-9eda-56f03edcd1ff,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,60.0000,tradable,call,2020-11-06T01:24:57.480675Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
8,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480630Z,2021-02-19,dc19d215-1a5d-4ad3-acd1-be58842f605f,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,77.5000,tradable,call,2020-11-06T01:24:57.480636Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
9,e66ce029-db96-4572-87a0-b144613c08bf,AMD,2020-11-06T01:24:57.480591Z,2021-02-19,f5c0f0af-5806-4988-b84e-a88d1c944bce,1987-01-12,"{'above_tick': '0.05', 'below_tick': '0.01', '...",untradable,active,87.5000,tradable,call,2020-11-06T01:24:57.480597Z,https://api.robinhood.com/options/instruments/...,2021-02-19T19:45:00+00:00,0.25
